Bij het implementeren van over- en undersampling als technieken voor data-augmentatie, is het belangrijk om verschillende methoden te begrijpen en voorzichtig te werk te gaan om te zorgen dat de resulterende dataset een evenwichtige weergave biedt zonder overfitting te introduceren. Hier is hoe je deze technieken kunt aanpakken:
 

### Oversampling
 

Oversampling houdt in dat je meer voorbeelden toevoegt aan de minderheidscategorieën om een evenwichtiger dataset te creëren. Hier zijn enkele methoden:
 
1. **Eenvoudige duplicatie**:
   - Kopieer willekeurig bestaande voorbeelden van de minderheidscategorie totdat de categorieën in balans zijn.
   - Voordeel: Eenvoudig en snel te implementeren.
   - Nadeel: Kan leiden tot overfitting doordat hetzelfde voorbeeld meerdere keren wordt gezien.
 
2. **SMOTE (Synthetic Minority Over-sampling Technique)**:
   - Deze techniek genereert synthetische voorbeelden door de kenmerken van nabijgelegen voorbeelden van de minderheidscategorie te interpoleren.
   - Voordeel: Creëert nieuwe, unieke voorbeelden waardoor overfitting wordt verminderd.
   - Nadeel: Compliceert de dataset met gegenereerde data die misschien niet volledig representatief is.
 
3. **ADASYN (Adaptive Synthetic Sampling)**:
   - Vergelijkbaar met SMOTE, maar richt zich meer op de moeilijker te classificeren voorbeelden van de minderheidscategorie.
   - Voordeel: Nadruk op uitdagendere gebieden kan de classifier robuuster maken.
   - Nadeel: Kan leiden tot het genereren van ruis als het niet goed wordt toegepast.
 

Het is verstandig oversampling of undersampling toe te passen voor je een eigen gemiddelde en standaardafwijking gaat berekenen aangezien de uitkomsten hiervan niet meer correct zijn voor de dataset nadat hier extra afbeeldingen aan zijn toegevoegd of bij weg zijn gehaald.

### Eenvoudige oversampling

Hier kopiëren we willekeurig voorbeelden van de minderheidscategorie om de dataset in balans te brengen.

In [6]:
import os
import shutil
import numpy as np
from sklearn.model_selection import train_test_split

In [7]:
# Stel de paden in voor de dataset
base_dir = '..\\..\\Data\\'
dataset_dir = base_dir + 'eigen_afbeeldingen_11_12'
oversampled_dataset_dir = base_dir + 'oversampled_dataset'
undersampled_dataset_dir = base_dir + 'undersampled_dataset'

# Creëer directories als ze nog niet bestaan
os.makedirs(oversampled_dataset_dir, exist_ok=True)
os.makedirs(undersampled_dataset_dir, exist_ok=True)

# Lijst van klassen (subdirectory-namen)
classes = os.listdir(dataset_dir)

In [8]:
classes

['Appel', 'Banaan', 'Kiwi', 'Peer', 'sinaasappel_mandarijn', 'Tomaat']

    Door data understanding weten we dat de class verdeling er zo uitziet - Voeg hier die tool toe

We hebben nu verschillende opties, we zouden de alles kunnen undersamplen naar de klasse van banaan, alles oversamplen naar sinaasappel_mandarijn of er tussening van appel. Aangezien dit proces wordt nu wordt voorgedaan kiezen we voor de laatste.

In [11]:
# Verkrijg de lijst van bestanden per klasse
files_banaan = os.listdir(os.path.join(dataset_dir, 'Banaan')) # Oversample
files_sinaasappel_mandarijn = os.listdir(os.path.join(dataset_dir, 'sinaasappel_mandarijn')) # Undersample
files_appel = os.listdir(os.path.join(dataset_dir, 'Appel')) # Target amount

In [12]:
# Voorbeeld voor eenvoudige oversampling door dupliceren van class_1 image bestanden
# Zorg ervoor dat class_1 in balans is met class_0 in terms of aantal samples
def simple_oversample_images(source_dir, dest_dir, num_samples_needed):
    source_files = os.listdir(source_dir)
    os.makedirs(dest_dir, exist_ok=True)
    
    # Kopieer alle bestaande bestanden
    for file in source_files:
        shutil.copy(os.path.join(source_dir, file), os.path.join(dest_dir, file))
        
    # Dupliceren van willekeurige bestanden tot we genoeg hebben
    additional_files = np.random.choice(source_files, num_samples_needed, replace=True)
    for i, file in enumerate(additional_files):
        new_file_name = f"dup_{i}_{file}"
        shutil.copy(os.path.join(source_dir, file), os.path.join(dest_dir, new_file_name))

In [13]:
# Oversample de minderheidscategorie
num_samples_needed = len(files_appel) - len(files_banaan)
simple_oversample_images(os.path.join(dataset_dir, 'Banaan'), os.path.join(oversampled_dataset_dir, 'Banaan'), num_samples_needed)

In [14]:
# Kopieer de meerderheidscategorie zonder verandering
shutil.copytree(os.path.join(dataset_dir, 'Appel'), os.path.join(oversampled_dataset_dir, 'Appel'))

'..\\..\\Data\\oversampled_dataset\\Appel'

    Voeg hier weer de understanding tool toe

We maken de keuze voor een eenvoudige oversampling doordat bij het inladen van de data hier nog random aanpassingen op worden gedaan waardoor geen exact zelfde afbeeldingen zullen voorkomen.

### Undersampling
 

Undersampling reduceert het aantal voorbeelden in de meerderheidscategorie om balans te bereiken. Hier zijn enkele methoden:
 
1. **Willekeurige ondersampling**:
   - Verwijder willekeurig voorbeelden uit de meerderheidscategorie totdat er balans is.
   - Voordeel: Gemakkelijk te implementeren en kan computationele belasting verminderen.
   - Nadeel: Verlies van potentiële informatie en kan leiden tot underfitting.
 
2. **Cluster-gebaseerde ondersampling**:
   - Gebruik clustering (zoals k-means) om de meerderheidscategorie in clusters te verdelen en dan voorbeelden te selecteren die het best elk cluster representeren.
   - Voordeel: Behoudt representativiteit binnen de meerderheidscategorie.
   - Nadeel: Complexer en tijdsintensiever.
 
3. **NearMiss**:
   - Strategieën zoals NearMiss 1, 2, of 3 selecteren voorbeelden op basis van hun afstand tot minderheidsvoorbeelden.
   - Voordeel: Helpt bij het leren van betere beslissingsgrenzen.
   - Nadeel: Kan computationeel intensief zijn.
 

### Willekeurige ondersampling

Hier verwijderen we willekeurig voorbeelden uit de meerderheidscategorie om de dataset in balans te brengen.

In [21]:
# Voorbeeld voor willekeurige ondersampling door verwijderen van sinaasappel_mandarijn image bestanden
def random_undersample_images(source_dir, dest_dir, num_samples_to_keep):
    source_files = os.listdir(source_dir)
    os.makedirs(dest_dir, exist_ok=True)
    
    # Selecteer willekeurig bestanden om te behouden
    selected_files = np.random.choice(source_files, num_samples_to_keep, replace=False)
    for file in selected_files:
        shutil.copy(os.path.join(source_dir, file), os.path.join(dest_dir, file))

In [22]:
# Undersample de meerderheidscategorie
random_undersample_images(os.path.join(dataset_dir, 'sinaasappel_mandarijn'), 
                          os.path.join(undersampled_dataset_dir, 'sinaasappel_mandarijn'), 
                          len(files_appel))

# Kopieer de minderheidscategorie zonder verandering
shutil.copytree(os.path.join(dataset_dir, 'Appel'), os.path.join(undersampled_dataset_dir, 'Appel'))

'..\\..\\Data\\undersampled_dataset\\Appel'

    Voeg hier weer de understanding tool toe

Zo kan je nu met bovenstaande code een dataset compleet gelijk maken.